![](pics\task1.jpg)

![](pics\task1_2.jpg)

In [1]:
import numpy as np
from numpy.random import default_rng

In [2]:
dr = default_rng(0)

In [3]:
def rand(a, b, m, n):
    return (b-a) * dr.random((m, n)) + a

def basicDistance(sample1, sample2):
    dist = np.linalg.norm(sample1 - sample2)

def distance(clast1, clast2):
    return (clast1.d + clast2.d) / 2



In [8]:
def basicDistanceMatrix(samples: np.array):
    prev_distances = np.zeros((samples.size, samples.size))
    for i in range(samples.size):
        for k in range(i + 1):
            prev_distances[i][k] = basicDistance(samples[i], samples[k])
            prev_distances[k][i] = prev_distances[i][k]

In [9]:
def clasterizing(samples: np.array):
    c = []
    for sample in samples:
        c.append(sample)
    prev_distances = basicDistanceMatrix(samples)
    print(prev_distances)
    return c 

    

In [11]:
arr = rand(1, 9, 10, 2)
print(arr)

[[1.22655737 1.99426621]
 [6.36499532 6.17751609]
 [5.92308089 4.06942043]
 [8.97767949 8.84668271]
 [6.48433588 6.20367421]
 [6.50757384 4.11137139]
 [2.08077204 6.77190672]
 [5.20283458 3.481935  ]
 [4.88668287 8.11590267]
 [8.47234813 3.86236157]]


In [12]:
C = clasterizing(arr)


IndexError: index 10 is out of bounds for axis 0 with size 10